In [2]:
#Analyzing Historical Stock/Revenue Data and Building a Dashboard

In [24]:
# =========================================================
# Peer-graded Assignment: Stocks, Revenue & Dashboards
# 100% yfinance solution (no scraping, no broken URLs)
# Colab-ready
# =========================================================

!pip -q install yfinance plotly

import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# ---------- Helpers ----------

def get_total_revenue_quarterly(ticker: str) -> pd.DataFrame:
    """
    Robustly extract quarterly Total Revenue for a given ticker using yfinance.
    Returns a DataFrame with columns: Date, Revenue (float), sorted by Date.
    """
    tk = yf.Ticker(ticker)

    # yfinance returns quarterly_financials with row index as line items,
    # columns are period end dates. We'll find a row that matches "Total Revenue".
    qfin = tk.quarterly_financials  # rows: metrics, cols: period end dates

    if qfin is None or qfin.empty:
        raise ValueError(f"No quarterly financials available for {ticker}")

    # Find the best-matching index label for "Total Revenue" (case/space-insensitive)
    def norm(s): return str(s).strip().lower().replace(" ", "")
    candidates = {idx: norm(idx) for idx in qfin.index}
    target = None
    for idx, key in candidates.items():
        if key in ("totalrevenue", "revenuetotal") or ("revenue" in key and "total" in key):
            target = idx
            break
    if target is None:
        # Fallback: pick the first index containing 'revenue'
        for idx, key in candidates.items():
            if "revenue" in key:
                target = idx
                break

    if target is None or target not in qfin.index:
        raise ValueError(f"Could not locate a 'Total Revenue' row in quarterly_financials for {ticker}.\nAvailable rows: {list(qfin.index)}")

    # Extract the series and convert to a tidy DataFrame
    ser = qfin.loc[target]           # index = period-end Timestamp, values = revenue
    df = ser.to_frame(name="Revenue").reset_index().rename(columns={"index": "Date"})
    # Ensure proper types & ordering
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)
    # Convert to float (values are usually numeric already)
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    df = df.dropna(subset=["Revenue"])
    return df

def make_graph(stock_data: pd.DataFrame, revenue_data: pd.DataFrame, stock_title: str, revenue_title: str):
    """
    Draw two Plotly line charts: stock close price and quarterly revenue.
    Expects:
      stock_data columns: Date, Close
      revenue_data columns: Date, Revenue
    """
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Close"], name="Stock Price"))
    fig1.update_layout(title=stock_title, xaxis_title="Date", yaxis_title="Price ($)")
    fig1.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=revenue_data["Date"], y=revenue_data["Revenue"], name="Revenue"))
    fig2.update_layout(title=revenue_title, xaxis_title="Date", yaxis_title="Revenue")
    fig2.show()

# ---------- Q1: Tesla stock data ----------
tesla_data = yf.download("TSLA", start="2010-01-01")
tesla_data.reset_index(inplace=True)   # required: reset index
display(tesla_data.head())             # 📸 Screenshot for Q1 (code + first 5 rows)

# ---------- Q2: Tesla revenue (from yfinance financials) ----------
tesla_revenue = get_total_revenue_quarterly("TSLA")
display(tesla_revenue.tail())          # 📸 Screenshot for Q2 (last 5 rows)

# ---------- Q3: GameStop stock data ----------
gme_data = yf.download("GME", start="2010-01-01")
gme_data.reset_index(inplace=True)     # required: reset index
display(gme_data.head())               # 📸 Screenshot for Q3 (code + first 5 rows)

# ---------- Q4: GameStop revenue (from yfinance financials) ----------
gme_revenue = get_total_revenue_quarterly("GME")
display(gme_revenue.tail())            # 📸 Screenshot for Q4 (last 5 rows)

# ---------- Q5: Tesla graphs ----------
make_graph(
    stock_data=tesla_data[["Date", "Close"]],
    revenue_data=tesla_revenue[["Date", "Revenue"]],
    stock_title="Tesla Stock Price Over Time",
    revenue_title="Tesla Quarterly Revenue"
)                                       # 📸 Screenshot the graph(s)

# ---------- Q6: GameStop graphs ----------
make_graph(
    stock_data=gme_data[["Date", "Close"]],
    revenue_data=gme_revenue[["Date", "Revenue"]],
    stock_title="GameStop Stock Price Over Time",
    revenue_title="GameStop Quarterly Revenue"
)                                       # 📸 Screenshot the graph(s)

# ---------- Q7: Helper to print your submission URL ----------
submission_url = "https://github.com/HarshithaTech/boston-housing-project"  # replace if you use a different repo
print("Submit this URL in Coursera:", submission_url)


/tmp/ipython-input-1064477313.py:76: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,TSLA,TSLA,TSLA,TSLA,TSLA
0,2010-06-29,1.592667,1.666667,1.169333,1.266667,281494500
1,2010-06-30,1.588667,2.028000,1.553333,1.719333,257806500
2,2010-07-01,1.464000,1.728000,1.351333,1.666667,123282000
3,2010-07-02,1.280000,1.540000,1.247333,1.533333,77097000
4,2010-07-06,1.074000,1.333333,1.055333,1.333333,103003500


,Date,Revenue
1,2024-06-30,2.550000e+10
2,2024-09-30,2.518200e+10
3,2024-12-31,2.570700e+10
4,2025-03-31,1.933500e+10
5,2025-06-30,2.249600e+10


/tmp/ipython-input-1064477313.py:85: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,GME,GME,GME,GME,GME
0,2010-01-04,3.854642,3.863059,3.703150,3.714932,26702800
1,2010-01-05,3.959005,3.996037,3.854643,3.856327,21269600
2,2010-01-06,4.044851,4.056634,3.908507,3.948905,21471200
3,2010-01-07,3.443930,3.563441,3.268872,3.368184,164761200
4,2010-01-08,3.415314,3.573540,3.405215,3.474228,47872400


,Date,Revenue
1,2024-07-31,7.983000e+08
2,2024-10-31,8.603000e+08
3,2025-01-31,1.282600e+09
4,2025-04-30,7.324000e+08
5,2025-07-31,9.722000e+08


Submit this URL in Coursera: https://github.com/HarshithaTech/boston-housing-project
